# Imports

In [1]:
import pickle
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
from sklearn.svm import SVC
from tensorflow import keras
from datetime import datetime
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import mpl_toolkits
from mpl_toolkits.mplot3d import Axes3D
from imblearn.over_sampling import SMOTE
from sklearn.dummy import DummyClassifier
from sklearn.metrics import roc_curve, auc
from keras import models, layers, optimizers
from matplotlib.animation import FuncAnimation
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV, cross_validate
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, \
accuracy_score, f1_score, roc_curve, auc

2024-10-24 22:51:09.531851: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-24 22:51:09.550468: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-24 22:51:09.555964: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-24 22:51:09.569646: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-24 22:51:10.533885: W tensorflow/compiler/tf2

# Data Understanding

In [2]:
pp_1 = pd.read_excel("utumishi.xlsx", sheet_name = 'PP1', skiprows = 1)
pp_1 = pp_1.loc[:, :'STATUS']
pp_1

,CLASS,ADM_NO,STUDENT_NAME,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS
0,PP1,1678,DYLAN MWAURA MUNGAI,D,Samuel Mwaura,0722961925,PP1 Ruby,TR Esther,Present
1,PP1,1687,ALBERT NJENGA KINENE,D,Joshua Kinene,NaN,PP1 Lily,TR Virginia,Present
2,PP1,1745,ETHAN WANJAU WANJIKU,D,NaN,NaN,NaN,NaN,NaN
3,PP1,1756,JOSEPH NDUNGU KARARI,D,Jonh Karari,NaN,NaN,TR Ruth,Present
4,PP1,1836,NELLY WAMBUI,D,Wilson Kamaneu,NaN,PP1 Ivory,TR Ruth,Present
...,...,...,...,...,...,...,...,...,...
141,PP1,12098,KRYSTAL NJERI MUNYWE,D,John Michau,0717178010,PP1 Ivory,Tr Ruth,Present
142,PP1,12129,ABIGAIL WAKARURA NJUGUNA,D,Henry Njunguna,0725026893,PP1,NaN,Present
143,PP1,12227,DARIAN NYORO KAGIA,D,NaN,NaN,NaN,NaN,NaN
144,PP1,12258,HENRY WAIGWA KIMANI,D,Ann Kimani,0710625552,PP1 Ruby,Tr Esther,Present


In [3]:
pp_2 = pd.read_excel("utumishi.xlsx", sheet_name = 'PP2', skipfooter = 1)
pp_2

,CLASS,ADM_NO,STUDENT_NAME,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS
0,PP2,1464.0,SAMMY KANYUA KIRIBA,D,Joseph Kiriba,0720670271,PP2 Lily,TR.Sarah,PRESENT
1,PP2,1524.0,RYAN KAMAU KUNGU(TR),D,NaN,NaN,NaN,NaN,NaN
2,PP2,1658.0,NANCY NJAMBI MWAURA,D,Miren Wanjiru,0798937252,PP2 Ruby,TR.Tabitha,PRESENT
3,PP2,1659.0,HOPE WANJA NDUATI,D,Wilson Ndungu,0724672975,PP2 Ivory,TR.Mary,PRESENT
4,PP2,1660.0,SHANNEL MARIE NJUGUNA,D,George Njuguna,0729445387,PP2 Lily,TR.Sarah,PRESENT
...,...,...,...,...,...,...,...,...,...
124,PP2,11890.0,KEYSHIA GATHONI NJERI,D,IRENE MUIRURI,0729682772,PP2 Ruby,TR.Tabitha,PRESENT
125,PP2,11891.0,NATASHA WAMBUI,D,WILLY THUITAI,0727457508,PP2 Lily,TR.Sarah,PRESENT
126,PP2,11894.0,KIMI NJERI NJOROGE,D,SIMON NJOROGE,0720857455,PP2 Ivory,TR.Mary,PRESENT
127,PP2,11905.0,LAQUEEN MBUTU,D,CAROLINE NJAHIRA,NaN,NaN,NaN,PRESENT


In [4]:
grade_1 = pd.read_excel("utumishi.xlsx", sheet_name = 'GRADE 1', skiprows = 1, skipfooter = 4)
grade_1

,CLASS,ADM_NO,STUDENT_NAME,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS,DOB
0,1,1269,Stephen Kibe,D,ANN KIBE,702120317.0,1 IVORY,TR. JOSEPHINE,PRESENT,17/5/2017
1,1,1270,Samuel Burugu,D,ANN KIBE,702120317.0,1 IVORY,TR. JOSEPHINE,PRESENT,17/5/2017
2,1,1386,Adrian Karuga Ngure,D,JOHN KARUGA,715675717.0,1 LILY,TR. RUTH,PRESENT,24/4/2017
3,1,1394,NATHANIEL NGUNYU MBOGO,D,GEORGE MBUGU,729208160.0,1 LILY,TR. RUTH,PRESENT,30/8/2017
4,1,1403,ANDREW ICHUGWA NYOKABI,D,CATHERINE WAMBUI,720308961.0,1 RUBY,TR. MARY,PRESENT,16/2/2018
...,...,...,...,...,...,...,...,...,...,...
176,1,12281,GRIFFIN KIPRUTO,D,VICTOR KIPROTICU,706427263.0,1 LILY,TR RUTH,PRESENT,2016-11-11 00:00:00
177,1,12282,PENDO LESLLEY,D,VIOLET AWINO,720116459.0,1 LILY,TR RUTH,PRESENT,19/4/2017
178,1,12283,BRIELLE WEMA.,D,JOB NDEGWA,721379291.0,1 LILY,TR RUTH,PRESENT,2017-11-04 00:00:00
179,1,12284,ANDREW TIBI,D,TERESIAH WAWERU,705286731.0,1 LILY,TR RUTH,PRESENT,23/4/2018


In [5]:
grade_2 = pd.read_excel("utumishi.xlsx", sheet_name = 'GRADE 2')
grade_2

,CLASS,ADM_NO,STUDENT_NAME,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS,DOB
0,2,1133.0,Joshua Njau,D,Jane Wangari Njau,729647358.0,2 Violet,TR Violet,Present,NaN
1,2,1230.0,Petitah Gachiku Njunge,D,Margret Wamburu,720389709.0,2 Ivory,TR Mary,Present,NaN
2,2,1236.0,Hannah wanjiru Wangechi,D,Sarah Wangechi,721857978.0,2 Lily,TR Josphine,Present,NaN
3,2,1237.0,Dylan ndungu Chege,D,NaN,NaN,NaN,NaN,NaN,NaN
4,2,1238.0,Everswine njeru Mwangi,D,Peter Mwangi,728808475.0,2 Violet,TR Violet,Present,NaN
...,...,...,...,...,...,...,...,...,...,...
204,2,NaN,Dorcas Njeri,D,David Njoroge,723494125.0,2 Lily,TR Josphine,Present,NaN
205,2,NaN,Adrian Kariuki,D,Serah Kariuki,729579813.0,2 Ivory,TR Mary,Present,NaN
206,2,NaN,Margaret Wangechi,NaN,Flaciah Wairimu,797920811.0,2 Lily,TR Josphine,Present,NaN
207,2,NaN,Ethan Gatimu,NaN,Carolyne Njenga,720326190.0,2 Lily,TR Josphine,Present,NaN


In [6]:
grade_3 = pd.read_excel("utumishi.xlsx", sheet_name = 'GRADE 3')
grade_3

,CLASS,ADM_NO,STUDENT_NAME,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS
0,3,1040.0,Owen Njau,D,TRANSFERED,NaN,NaN,NaN,Transferred
1,3,1123.0,Lucy Wairimu,D,MARY NJOKI,725458705.0,3 IVORY,TR TERESIAH,PRESENT
2,3,1124.0,Lewis Githogori,D,LILIAN GACHOGO,721362093.0,3 IVORY,TR TERESIAH,PRESENT
3,3,1125.0,Ellah Murugi,D,SIMON WAIRI,722454418.0,3 VIOLET,TR JUDY,PRESENT
4,3,1126.0,Dylan Davis Macharia,D,CHEGE MBUGUA,720572731.0,3 IVORY,TR TERESIAH,PRESENT
...,...,...,...,...,...,...,...,...,...
170,3,11983.0,TREACY WAMBUI,D,PAUL NJOROGE,727822631.0,3 RUBY,TR EUNICE,PRESENT
171,3,1207.0,MALIK MWANGI,D,VERONICA WANGU,708853698.0,3 RUBY,TR EUNICE,PRESENT
172,3,12076.0,JAYDEN KARIUKI,D,MONICA NJENGA,728473002.0,3 RUBY,TR EUNICE,PRESENT
173,3,NaN,VIVIAN WANJIRU,D,LUCIA NJOKI,721282419.0,3 LILY,TR ALICE,PRESENT


In [7]:
grade_4 = pd.read_excel("utumishi.xlsx", sheet_name = 'GRADE 4')
grade_4

,CLASS,ADM_NO,STUDENT_NAME,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS
0,4,1780,JAYDEN PETER KIN,D,Irene Njeri,0729682772,4 Violet,Tr. Monica,Present
1,4,11173,Cynthia Nyambura Kinyanjui,D,Beatrice Wambui,0746794822,4 Lily,Tr. Emma,Present
2,4,11216,Wayne Travis,D,Charles Muchuri,NaN,4 Ruby,Tr. Carol,Present
3,4,11297,Bracha Wangari,D,Willfred Mwangi,0707419270,4 Lily,Tr. Emma,Present
4,4,11298,David Chege,D,NaN,NaN,NaN,NaN,TRANSFERRED
...,...,...,...,...,...,...,...,...,...
192,4,11957,DAVID WAHOME,D,JOSEPHAT GICHUKI,0720792628,4 Ivory,Mr.Asiago,Present
193,4,11958,HARRY KIMANI,D,EDWARD KURIA,0725773991,4 Ivory,Mr.Asiago,Present
194,4,11959,ANGELA WANJIRU,D,DAMARIS WANYIKA,0713894684,4 Ivory,Mr.Asiago,Present
195,4,11960,TANYA BLESSING,D,Richard Chege,0724527746,4 Ivory,Mr.Asiago,Present


In [8]:
grade_5 = pd.read_excel("utumishi.xlsx", sheet_name = 'GRADE 5')
grade_5

,CLASS,ADM_NO,STUDENT_NAME,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS
0,5,11114,Daisy Muthoni,D,NaN,NaN,NaN,NaN,TRANSFERED
1,5,11115,Linda Wanjiku,D,Evanson Kimemia,721809667.0,5 Violet,TR victor,Present
2,5,11116,Sammy Macharia,D,Gilbert Kamau,724571099.0,5 Lily,TR Faith,Present
3,5,11117,Samuel Kamanu,D,John Paul,708632700.0,5 Violet,TR victor,Present
4,5,11118,Jayden John Kariuki,D,Phyllis Njoki,722106400.0,5 Ruby,TR Esther,Present
...,...,...,...,...,...,...,...,...,...
186,5,11995,TATTYANA NJOKI,D,Nicholas Mwangi,721860414.0,5 Lily,Tr.Faith,Present
187,5,11996,MARY NJOKI,D,John Karari,725883885.0,5 Lily,Tr.Faith,Present
188,5,11997,MARTIN MATI,D,Paul Gatere,722889868.0,5 Lily,Tr.Faith,Present
189,5,11998,SANDRA NJAMBI,D,David Thiga,729098649.0,5 Lily,Tr.Faith,Present


In [9]:
grade_6 = pd.read_excel("utumishi.xlsx", sheet_name = 'GRADE 6')
grade_6

,CLASS,ADM_NO,STUDENT_NAME,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS
0,6,10756,Joseph Muthinji,D,Benson Mbugua,722253119.0,6 Violet,Mr. Chege,Present
1,6,10981,Aimee Muthoni,D,George Kigotho,722486275.0,6 Ivory,Tr. Michael,Present
2,6,10982,Winner Munene,D,Peter Ng`ang`a,721539840.0,6Lily,Tr. John,Present
3,6,10983,Mish Samuel Njoroge,D,Kate Njoki,710731236.0,6 Ivory,Tr. Michael,Present
4,6,10984,Favour Njambi,B,Samuel Njenga,720532842.0,6Lily,Tr. John,Present
...,...,...,...,...,...,...,...,...,...
144,6,11876,JOSEPH MUNGAI WANJIRU,B,Wanjiru Mungai,717106443.0,6 Ivory,Tr. Michael,Present
145,6,11925,TRAVIS MUCHUGIA KABUU,D,Jane Waithera,720732231.0,6 Ruby,Tr. Njeri,Present
146,6,11927,MAXWELL NJOROGE,B,Eunice Wanjiru,722992517.0,6 Ivory,Tr. Michael,Present
147,6,11935,ROMNEY KYAMBU,D,Mercy Nyokabi,724940922.0,6 Ivory,Tr. Michael,Present


In [10]:
grade_7 = pd.read_excel("Updated grade 7& 8.xlsx", skiprows = 2, sheet_name = '7')
grade_7 = grade_7.drop(columns=['Sno'])
grade_7

,ADM_NO,STUDENT_NAME,CLASS,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS,DOB,GENDER
0,254,Victor Kiruri,7,B,Richard Mbugua,722450097,7 Tana,TR.Emmah,Present,22/10/2011,M
1,255,Emmanuella Baraka,7,B,Meshack Kiarie,725809292,7Tana,TR.Emmah,Present,23/10/2011,F
2,256,Precious Ngendo,7,D.,Michael Ng'ang'a,721963525,7 Nile,Mr.Mwangi,Present,4/2/2012,F
3,257,Krisha Njeri,7,D,Justin Mwangi,721321327,7 Athi,TR.Lydia,Present,15/4/2011,F
4,258,Victor Kimani,7,D,Simon Kihara,721471568,7 Chania,Mr.Ng'ang'a,Present,8/3/2012,M
...,...,...,...,...,...,...,...,...,...,...,...
251,543,KEISHA THAMI GAKO,7,B,Amos Gakuo,726373228,7 Tana,Tr. Emmah,Present,29/4/2011,F
252,545,JABARI GAKUNJU,7,B,Caroline Karanja,726477463,7 Chania,Mr. Ng'ang'a,Present,6/11/2010,M
253,546,KESSY KIMBERLY NEEMA,7,D,Violet Nelly,720116459,7 Tana,TR.Emmah,Present,25/5/2012,F
254,516,MARK P. KABACHIA,7,B,Bernard Kiratu,721663785,7 Chania,Mr.Ng'ang'a,Present,15/1/2012,M


In [11]:
grade_8 = pd.read_excel("Updated grade 7& 8.xlsx", skiprows = 1, sheet_name = '8')
grade_8

,ADM_NO,STUDENT_NAME,CLASS,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS,DOB
0,1,Kathy Waringa,8,B,Timothy Ndegwa,721816708.0,8 Chania,TR.Vivian,Present,14/6/2010
1,2,Winfrey Wangari,8,B,Rebecca Wanjiru,726369053.0,8 Galana,Mr. Opiyo,Present,13/9/2010
2,3,Ian Njoroge Njuguna,8,B,Ruth Wanjiku,721817755.0,8 Chania,TR.Vivian,Present,2010-02-02 00:00:00
3,4,Rosemary Wanjiru,8,B,Peter Ndungu,721467810.0,8 Athi,TR.David,Present,22/2/2011
4,5,Eric Kariuki,8,B,Mary Wanjiku,720246830.0,8 Nile,Mr.Wanjala,Present,15/3/2010
...,...,...,...,...,...,...,...,...,...,...
230,541,LEWIS NGARU WACHIRA,8,B,George Gichimo,726261305.0,8 Chania,TR.Vivian,Present,16/4/2010
231,542,JAIME MUTHIORA MWANGI,8,B,David Mwangi,721426072.0,8 Galana,Mr Opiyo,Present,27/10/2011
232,544,JOEL CHAHILU CHALA,8,B,Jackeline Luvanda,723100910.0,8 Chania,TR.Vivian,Present,29/6/2010
233,245,STELLA NYAMBURA MUNGAI,8,B,Geoffrey Mungai,725753266.0,8 Galana,Mr Opiyo,Present,23/1/2011


In [12]:
dataframes_to_concat = [pp_1, pp_2, grade_1, grade_2, grade_3, grade_4, grade_5, grade_6, grade_7, grade_8]

schoolerp = pd.concat(dataframes_to_concat, ignore_index=True)
schoolerp

,CLASS,ADM_NO,STUDENT_NAME,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS,DOB,GENDER
0,PP1,1678.0,DYLAN MWAURA MUNGAI,D,Samuel Mwaura,0722961925,PP1 Ruby,TR Esther,Present,NaN,NaN
1,PP1,1687.0,ALBERT NJENGA KINENE,D,Joshua Kinene,NaN,PP1 Lily,TR Virginia,Present,NaN,NaN
2,PP1,1745.0,ETHAN WANJAU WANJIKU,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PP1,1756.0,JOSEPH NDUNGU KARARI,D,Jonh Karari,NaN,NaN,TR Ruth,Present,NaN,NaN
4,PP1,1836.0,NELLY WAMBUI,D,Wilson Kamaneu,NaN,PP1 Ivory,TR Ruth,Present,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1863,8,541.0,LEWIS NGARU WACHIRA,B,George Gichimo,726261305.0,8 Chania,TR.Vivian,Present,16/4/2010,NaN
1864,8,542.0,JAIME MUTHIORA MWANGI,B,David Mwangi,721426072.0,8 Galana,Mr Opiyo,Present,27/10/2011,NaN
1865,8,544.0,JOEL CHAHILU CHALA,B,Jackeline Luvanda,723100910.0,8 Chania,TR.Vivian,Present,29/6/2010,NaN
1866,8,245.0,STELLA NYAMBURA MUNGAI,B,Geoffrey Mungai,725753266.0,8 Galana,Mr Opiyo,Present,23/1/2011,NaN


In [13]:
schoolerp.columns

Index(['CLASS', 'ADM_NO', 'STUDENT_NAME', 'D/B', 'PARENT_NAME', 'TELEPHONE',
       'STREAM', 'CLASSTEACHER', 'STATUS', 'DOB', 'GENDER'],
      dtype='object')

In [14]:
schoolerp.head(10)

,CLASS,ADM_NO,STUDENT_NAME,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS,DOB,GENDER
0,PP1,1678.0,DYLAN MWAURA MUNGAI,D,Samuel Mwaura,0722961925,PP1 Ruby,TR Esther,Present,NaN,NaN
1,PP1,1687.0,ALBERT NJENGA KINENE,D,Joshua Kinene,NaN,PP1 Lily,TR Virginia,Present,NaN,NaN
2,PP1,1745.0,ETHAN WANJAU WANJIKU,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PP1,1756.0,JOSEPH NDUNGU KARARI,D,Jonh Karari,NaN,NaN,TR Ruth,Present,NaN,NaN
4,PP1,1836.0,NELLY WAMBUI,D,Wilson Kamaneu,NaN,PP1 Ivory,TR Ruth,Present,NaN,NaN
5,PP1,1837.0,LIAM KIMARI,D,Christopher Wamae,NaN,PP1 Lily,TR Virginia,Present,NaN,NaN
6,PP1,1838.0,OLIVE NJERI GICHUHI,D,Goefrey Gichuhi,0739838349,PP1 Ivory,TR Ruth,Present,NaN,NaN
7,PP1,1839.0,TREVIS NJUNGE,D,Paul Wakanya,NaN,PP1 Lily,TR Virginia,Present,NaN,NaN
8,PP1,1840.0,TRAVIS MBAU,D,James Mwangi,NaN,PP1 Ivory,TR Ruth,Present,NaN,NaN
9,PP1,1841.0,GRACE WAHU KANJA,D,Peter Kanja,0797173304,PP1 Lily,TR Virginia,Present,NaN,NaN


In [15]:
schoolerp.tail(10)

,CLASS,ADM_NO,STUDENT_NAME,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS,DOB,GENDER
1858,8,526.0,EUNICE WANJA GITU,B,Simon Gitu,726650974.0,8 Nile,MR.Wanjala,Present,30/7/2010,NaN
1859,8,532.0,FRANK KIBUGI WAMBU,B,Anthony Wambu,722746250.0,8 Tana,TR.Isaac,Present,2010-07-07 00:00:00,NaN
1860,8,533.0,TABITHA WAMBUI KIMATA,B,Elishiba Wangui,724620037.0,8 Tana,TR.Isaac,Present,2011-08-10 00:00:00,NaN
1861,8,534.0,KENNEDY KAMANGA GICHOHI,B,Hannah Muthoni,721570134.0,8 Nile,Mr.Wanjala,Present,28/8/2010,NaN
1862,8,539.0,SOPHIA WANJIRU NGANGA,B,Lucy Wanja,727127455.0,8 Galana,Mr Opiyo,Present,2011-01-02 00:00:00,NaN
1863,8,541.0,LEWIS NGARU WACHIRA,B,George Gichimo,726261305.0,8 Chania,TR.Vivian,Present,16/4/2010,NaN
1864,8,542.0,JAIME MUTHIORA MWANGI,B,David Mwangi,721426072.0,8 Galana,Mr Opiyo,Present,27/10/2011,NaN
1865,8,544.0,JOEL CHAHILU CHALA,B,Jackeline Luvanda,723100910.0,8 Chania,TR.Vivian,Present,29/6/2010,NaN
1866,8,245.0,STELLA NYAMBURA MUNGAI,B,Geoffrey Mungai,725753266.0,8 Galana,Mr Opiyo,Present,23/1/2011,NaN
1867,8,538.0,ARSENE GATERU THUKU,B,Hillary Thuku,721527865.0,8 Galana,Mr Opiyo,Present,2011-09-02 00:00:00,NaN


In [16]:
schoolerp.info

<bound method DataFrame.info of      CLASS  ADM_NO             STUDENT_NAME D/B        PARENT_NAME  \
0      PP1  1678.0      DYLAN MWAURA MUNGAI   D      Samuel Mwaura   
1      PP1  1687.0     ALBERT NJENGA KINENE   D      Joshua Kinene   
2      PP1  1745.0     ETHAN WANJAU WANJIKU   D                NaN   
3      PP1  1756.0     JOSEPH NDUNGU KARARI   D        Jonh Karari   
4      PP1  1836.0             NELLY WAMBUI   D     Wilson Kamaneu   
...    ...     ...                      ...  ..                ...   
1863     8   541.0      LEWIS NGARU WACHIRA   B     George Gichimo   
1864     8   542.0    JAIME MUTHIORA MWANGI   B       David Mwangi   
1865     8   544.0       JOEL CHAHILU CHALA   B  Jackeline Luvanda   
1866     8   245.0  STELLA NYAMBURA  MUNGAI   B    Geoffrey Mungai   
1867     8   538.0      ARSENE GATERU THUKU   B      Hillary Thuku   

        TELEPHONE     STREAM CLASSTEACHER   STATUS                  DOB GENDER  
0      0722961925   PP1 Ruby    TR Esther  Pre

In [17]:
schoolerp.describe()

,ADM_NO
count,1844.000000
mean,5082.688178
std,5173.845725
min,1.000000
25%,512.750000
50%,1698.500000
75%,11338.250000
max,12285.000000


In [18]:
schoolerp.shape

(1868, 11)

# Data Cleaning

## Checking for duplicates

In [19]:
schoolerp.duplicated().sum()

0

## Checking for Placeholders

In [20]:
# checking for placeholders
place_holder = ['\\#',',','/','-','@','\\*','_']

# for loop to loop through the columns checking for any placeholder
for placeholder in schoolerp.columns:
    if schoolerp[placeholder].dtype == 'object':
        for hold in place_holder:
            if schoolerp[placeholder].str.contains(hold).any():
                print(f"We have found place holders, '{hold}' in column '{placeholder}' ")

We have found place holders, '\*' in column 'TELEPHONE' 
We have found place holders, '/' in column 'DOB' 


## Dealing with Placeholders

In [21]:
schoolerp['TELEPHONE']= schoolerp['TELEPHONE'].str.replace('*','')
schoolerp['DOB']= schoolerp['DOB'].str.replace('/','')

In [22]:
# checking for placeholders
place_holder = ['\\#',',','/','-','@','\\*','_']

# for loop to loop through the columns checking for any placeholder
for placeholder in schoolerp.columns:
    if schoolerp[placeholder].dtype == 'object':
        for hold in place_holder:
            if schoolerp[placeholder].str.contains(hold).any():
                print(f"We have found place holders, '{hold}' in column '{placeholder}' ")

## Checking for Whitespace

In [23]:
schoolerp_whitespace = [col for col in schoolerp.columns if col.strip() != col]
print("Columns with whitespace:", schoolerp_whitespace)

Columns with whitespace: []


## Checking for Missing Values

In [24]:
schoolerp.isna().sum()

CLASS              0
ADM_NO            24
STUDENT_NAME       0
D/B                3
PARENT_NAME       90
TELEPHONE       1507
STREAM           120
CLASSTEACHER     119
STATUS            64
DOB             1393
GENDER          1742
dtype: int64

## Dealing with Missing Values

DOB and GENDER have way too many missing values so we drop them

In [25]:
schoolerp.drop(columns = ["DOB", "GENDER"], axis = 1, inplace = True)
schoolerp

,CLASS,ADM_NO,STUDENT_NAME,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS
0,PP1,1678.0,DYLAN MWAURA MUNGAI,D,Samuel Mwaura,0722961925,PP1 Ruby,TR Esther,Present
1,PP1,1687.0,ALBERT NJENGA KINENE,D,Joshua Kinene,NaN,PP1 Lily,TR Virginia,Present
2,PP1,1745.0,ETHAN WANJAU WANJIKU,D,NaN,NaN,NaN,NaN,NaN
3,PP1,1756.0,JOSEPH NDUNGU KARARI,D,Jonh Karari,NaN,NaN,TR Ruth,Present
4,PP1,1836.0,NELLY WAMBUI,D,Wilson Kamaneu,NaN,PP1 Ivory,TR Ruth,Present
...,...,...,...,...,...,...,...,...,...
1863,8,541.0,LEWIS NGARU WACHIRA,B,George Gichimo,NaN,8 Chania,TR.Vivian,Present
1864,8,542.0,JAIME MUTHIORA MWANGI,B,David Mwangi,NaN,8 Galana,Mr Opiyo,Present
1865,8,544.0,JOEL CHAHILU CHALA,B,Jackeline Luvanda,NaN,8 Chania,TR.Vivian,Present
1866,8,245.0,STELLA NYAMBURA MUNGAI,B,Geoffrey Mungai,NaN,8 Galana,Mr Opiyo,Present


D/B

In [26]:
db_rows = schoolerp[schoolerp["D/B"].isna()]
db_rows

,CLASS,ADM_NO,STUDENT_NAME,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS
662,2,NaN,Margaret Wangechi,NaN,Flaciah Wairimu,NaN,2 Lily,TR Josphine,Present
663,2,NaN,Ethan Gatimu,NaN,Carolyne Njenga,NaN,2 Lily,TR Josphine,Present
664,2,NaN,Shanice Zawadi,NaN,Theresia W.Thomas,NaN,2 Lily,TR Josphine,Present


Replacing them with D because all grade 2 students are day scholars

In [27]:
schoolerp["D/B"].fillna("D", inplace=True)

db_rows = schoolerp[schoolerp["D/B"].isna()]
db_rows

,CLASS,ADM_NO,STUDENT_NAME,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS


ADM_NO

In [28]:
ADM_NO_rows = schoolerp[schoolerp["ADM_NO"].isna()].head(10)
ADM_NO_rows

,CLASS,ADM_NO,STUDENT_NAME,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS
274,PP2,NaN,JOSHUA NDUNGU,D,GATHONI NDUNG'U,0722483859,PP2 Ivory,TR.Mary,PRESENT
654,2,NaN,FAITH WANJIRU,D,Sarah Nduta,NaN,NaN,NaN,NaN
655,2,NaN,GRAYSON KOMU,D,Kiarie Komu,NaN,2 Ruby,TR Susan,Present
656,2,NaN,JOY MUTHONI,D,Cathrine Wahiga,NaN,2 Ruby,TR Susan,Present
657,2,NaN,INNOCENT OTIENO,D,David Ochieng,NaN,2 Violet,TR Violet,Present
658,2,NaN,RYAN CASET,D,Margret Nyambura,NaN,2 Ivory,TR Mary,Present
659,2,NaN,NATALIE MUTHONI,D,Mary Ndung'u,NaN,2 Ivory,TR Mary,Present
660,2,NaN,Dorcas Njeri,D,David Njoroge,NaN,2 Lily,TR Josphine,Present
661,2,NaN,Adrian Kariuki,D,Serah Kariuki,NaN,2 Ivory,TR Mary,Present
662,2,NaN,Margaret Wangechi,D,Flaciah Wairimu,NaN,2 Lily,TR Josphine,Present


Giving all students missing admission numbers sequential numbers from 1837 onwards since 1837 is the last student having an admission number is 1836

In [29]:
ADM_NO_rows = schoolerp[schoolerp["ADM_NO"].isna()]

num_missing = ADM_NO_rows.shape[0]

# Generate sequential numbers starting from 1837
sequential_numbers = range(1837, 1837 + num_missing)

# Fill the missing ADM_NO values with these sequential numbers
schoolerp.loc[schoolerp["ADM_NO"].isna(), "ADM_NO"] = sequential_numbers

In [30]:
ADM_NO_rows

,CLASS,ADM_NO,STUDENT_NAME,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS
274,PP2,NaN,JOSHUA NDUNGU,D,GATHONI NDUNG'U,0722483859,PP2 Ivory,TR.Mary,PRESENT
654,2,NaN,FAITH WANJIRU,D,Sarah Nduta,NaN,NaN,NaN,NaN
655,2,NaN,GRAYSON KOMU,D,Kiarie Komu,NaN,2 Ruby,TR Susan,Present
656,2,NaN,JOY MUTHONI,D,Cathrine Wahiga,NaN,2 Ruby,TR Susan,Present
657,2,NaN,INNOCENT OTIENO,D,David Ochieng,NaN,2 Violet,TR Violet,Present
658,2,NaN,RYAN CASET,D,Margret Nyambura,NaN,2 Ivory,TR Mary,Present
659,2,NaN,NATALIE MUTHONI,D,Mary Ndung'u,NaN,2 Ivory,TR Mary,Present
660,2,NaN,Dorcas Njeri,D,David Njoroge,NaN,2 Lily,TR Josphine,Present
661,2,NaN,Adrian Kariuki,D,Serah Kariuki,NaN,2 Ivory,TR Mary,Present
662,2,NaN,Margaret Wangechi,D,Flaciah Wairimu,NaN,2 Lily,TR Josphine,Present


STATUS

In [31]:
STATUS_rows = schoolerp[schoolerp["STATUS"].isna()]
STATUS_rows

,CLASS,ADM_NO,STUDENT_NAME,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS
2,PP1,1745.0,ETHAN WANJAU WANJIKU,D,NaN,NaN,NaN,NaN,NaN
13,PP1,1845.0,IAN MAINA NJAU,D,Michael Njaau,NaN,PP1 Lily,TR Virginia,NaN
15,PP1,1847.0,IVY NJERI NJAAGA,D,David Njaaga,NaN,PP1 Lily,TR Virginia,NaN
16,PP1,1848.0,ALICE MBAIRE NJUNGE,D,NaN,NaN,NaN,NaN,NaN
18,PP1,1850.0,RYAN MWAURA NDUNGU,D,TRANSFERED,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
633,2,11838.0,ADRIEL MBURU GATHIRA,D,Joy Karimi,NaN,NaN,NaN,NaN
635,2,11840.0,CHLOE WANJIKU KINUTHIA,D,Georgina Njeri,NaN,NaN,NaN,NaN
647,2,11874.0,EDWARDJONZ NJUKI IREGI,D,NaN,NaN,NaN,NaN,NaN
654,2,1838.0,FAITH WANJIRU,D,Sarah Nduta,NaN,NaN,NaN,NaN


Fill status with Present since he/she has an admission number

In [32]:
schoolerp["STATUS"].fillna("PRESENT", inplace=True)

status_rows = schoolerp[schoolerp["STATUS"].isna()]
status_rows

,CLASS,ADM_NO,STUDENT_NAME,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS


Telephone

In [33]:
TELEPHONE_rows = schoolerp[schoolerp["TELEPHONE"].isna()]
TELEPHONE_rows

,CLASS,ADM_NO,STUDENT_NAME,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS
1,PP1,1687.0,ALBERT NJENGA KINENE,D,Joshua Kinene,NaN,PP1 Lily,TR Virginia,Present
2,PP1,1745.0,ETHAN WANJAU WANJIKU,D,NaN,NaN,NaN,NaN,PRESENT
3,PP1,1756.0,JOSEPH NDUNGU KARARI,D,Jonh Karari,NaN,NaN,TR Ruth,Present
4,PP1,1836.0,NELLY WAMBUI,D,Wilson Kamaneu,NaN,PP1 Ivory,TR Ruth,Present
5,PP1,1837.0,LIAM KIMARI,D,Christopher Wamae,NaN,PP1 Lily,TR Virginia,Present
...,...,...,...,...,...,...,...,...,...
1863,8,541.0,LEWIS NGARU WACHIRA,B,George Gichimo,NaN,8 Chania,TR.Vivian,Present
1864,8,542.0,JAIME MUTHIORA MWANGI,B,David Mwangi,NaN,8 Galana,Mr Opiyo,Present
1865,8,544.0,JOEL CHAHILU CHALA,B,Jackeline Luvanda,NaN,8 Chania,TR.Vivian,Present
1866,8,245.0,STELLA NYAMBURA MUNGAI,B,Geoffrey Mungai,NaN,8 Galana,Mr Opiyo,Present


In [34]:
schoolerp["TELEPHONE"].fillna("0", inplace=True)

telephone_rows = schoolerp[schoolerp["TELEPHONE"].isna()]
telephone_rows

,CLASS,ADM_NO,STUDENT_NAME,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS


STREAM

In [35]:
STREAM_rows = schoolerp[schoolerp["STREAM"].isna()]
STREAM_rows

,CLASS,ADM_NO,STUDENT_NAME,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS
2,PP1,1745.0,ETHAN WANJAU WANJIKU,D,NaN,0,NaN,NaN,PRESENT
3,PP1,1756.0,JOSEPH NDUNGU KARARI,D,Jonh Karari,0,NaN,TR Ruth,Present
16,PP1,1848.0,ALICE MBAIRE NJUNGE,D,NaN,0,NaN,NaN,PRESENT
18,PP1,1850.0,RYAN MWAURA NDUNGU,D,TRANSFERED,0,NaN,NaN,PRESENT
24,PP1,1856.0,BRAYSON MUKOMA NGUGI,D,NaN,0,NaN,NaN,PRESENT
...,...,...,...,...,...,...,...,...,...
1253,6,11009.0,Olivia Wanjiku Njoi,D,NaN,0,NaN,NaN,Transferred
1254,6,11010.0,Victor Njega,D,NaN,0,NaN,NaN,Transferred
1295,6,11059.0,George Kamau,D,NaN,0,NaN,Mr. Chege,Present
1313,6,11089.0,Tonia Mwago,D,NaN,0,NaN,NaN,Transferred


Fill with unknown since there is a section in the dashboard that "unknown" can be added up

In [36]:
schoolerp["STREAM"].fillna("UNKNOWN", inplace=True)

STREAM_rows = schoolerp[schoolerp["STREAM"].isna()]
STREAM_rows

,CLASS,ADM_NO,STUDENT_NAME,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS


PARENT NAME

In [37]:
PARENT_rows = schoolerp[schoolerp["PARENT_NAME"].isna()]
PARENT_rows

,CLASS,ADM_NO,STUDENT_NAME,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS
2,PP1,1745.0,ETHAN WANJAU WANJIKU,D,NaN,0,UNKNOWN,NaN,PRESENT
16,PP1,1848.0,ALICE MBAIRE NJUNGE,D,NaN,0,UNKNOWN,NaN,PRESENT
24,PP1,1856.0,BRAYSON MUKOMA NGUGI,D,NaN,0,UNKNOWN,NaN,PRESENT
29,PP1,1861.0,JANET NJOKI NJOROGE,D,NaN,0,UNKNOWN,NaN,PRESENT
72,PP1,1906.0,LEEVAN TREVOR NGANGA,D,NaN,0720857455,UNKNOWN,NaN,Present
...,...,...,...,...,...,...,...,...,...
1254,6,11010.0,Victor Njega,D,NaN,0,UNKNOWN,NaN,Transferred
1295,6,11059.0,George Kamau,D,NaN,0,UNKNOWN,Mr. Chege,Present
1313,6,11089.0,Tonia Mwago,D,NaN,0,UNKNOWN,NaN,Transferred
1317,6,11094.0,Charles Ngatia,D,NaN,0,UNKNOWN,NaN,Transferred


In [38]:
schoolerp["PARENT_NAME"].fillna("UNKNOWN", inplace=True)

PARENT_rows = schoolerp[schoolerp["PARENT_NAME"].isna()]
PARENT_rows

,CLASS,ADM_NO,STUDENT_NAME,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS


Class Teacher

In [39]:
Class_Teacher_rows = schoolerp[schoolerp["CLASSTEACHER"].isna()]
Class_Teacher_rows

,CLASS,ADM_NO,STUDENT_NAME,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS
2,PP1,1745.0,ETHAN WANJAU WANJIKU,D,UNKNOWN,0,UNKNOWN,NaN,PRESENT
16,PP1,1848.0,ALICE MBAIRE NJUNGE,D,UNKNOWN,0,UNKNOWN,NaN,PRESENT
18,PP1,1850.0,RYAN MWAURA NDUNGU,D,TRANSFERED,0,UNKNOWN,NaN,PRESENT
24,PP1,1856.0,BRAYSON MUKOMA NGUGI,D,UNKNOWN,0,UNKNOWN,NaN,PRESENT
29,PP1,1861.0,JANET NJOKI NJOROGE,D,UNKNOWN,0,UNKNOWN,NaN,PRESENT
...,...,...,...,...,...,...,...,...,...
1163,5,11645.0,HARMONY WANJIRU,D,UNKNOWN,0,UNKNOWN,NaN,TRANSFERRED
1253,6,11009.0,Olivia Wanjiku Njoi,D,UNKNOWN,0,UNKNOWN,NaN,Transferred
1254,6,11010.0,Victor Njega,D,UNKNOWN,0,UNKNOWN,NaN,Transferred
1313,6,11089.0,Tonia Mwago,D,UNKNOWN,0,UNKNOWN,NaN,Transferred


In [40]:
schoolerp["CLASSTEACHER"].fillna("UNKNOWN", inplace=True)

CLASSTEACHER_rows = schoolerp[schoolerp["CLASSTEACHER"].isna()]
CLASSTEACHER_rows

,CLASS,ADM_NO,STUDENT_NAME,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS


## Confirming there are no missing values

In [41]:
schoolerp.isna().sum()

CLASS           0
ADM_NO          0
STUDENT_NAME    0
D/B             0
PARENT_NAME     0
TELEPHONE       0
STREAM          0
CLASSTEACHER    0
STATUS          0
dtype: int64

# DataFrame after data cleaning

In [42]:
schoolerp

,CLASS,ADM_NO,STUDENT_NAME,D/B,PARENT_NAME,TELEPHONE,STREAM,CLASSTEACHER,STATUS
0,PP1,1678.0,DYLAN MWAURA MUNGAI,D,Samuel Mwaura,0722961925,PP1 Ruby,TR Esther,Present
1,PP1,1687.0,ALBERT NJENGA KINENE,D,Joshua Kinene,0,PP1 Lily,TR Virginia,Present
2,PP1,1745.0,ETHAN WANJAU WANJIKU,D,UNKNOWN,0,UNKNOWN,UNKNOWN,PRESENT
3,PP1,1756.0,JOSEPH NDUNGU KARARI,D,Jonh Karari,0,UNKNOWN,TR Ruth,Present
4,PP1,1836.0,NELLY WAMBUI,D,Wilson Kamaneu,0,PP1 Ivory,TR Ruth,Present
...,...,...,...,...,...,...,...,...,...
1863,8,541.0,LEWIS NGARU WACHIRA,B,George Gichimo,0,8 Chania,TR.Vivian,Present
1864,8,542.0,JAIME MUTHIORA MWANGI,B,David Mwangi,0,8 Galana,Mr Opiyo,Present
1865,8,544.0,JOEL CHAHILU CHALA,B,Jackeline Luvanda,0,8 Chania,TR.Vivian,Present
1866,8,245.0,STELLA NYAMBURA MUNGAI,B,Geoffrey Mungai,0,8 Galana,Mr Opiyo,Present


# EDA